## Import Libraries

In [30]:
# !pip install python-Levenshtein
!pip install webcolors

In [0]:
import os
import sys
import operator
import PIL
from io import BytesIO
import tarfile
import tempfile
from six.moves import urllib
import pandas as pd
from matplotlib import gridspec
from matplotlib import pyplot as plt
import numpy as np
from PIL import Image
from functools import reduce
import tensorflow as tf
from sklearn.cluster import KMeans
from scipy import spatial
# import Levenshtein as leven
import webcolors
import json
import cv2

## Import helper methods
These methods help us perform the following tasks:
* Load the latest version of the pretrained DeepLab model
* Load the colormap from the PASCAL VOC dataset
* Adds colors to various labels, such as "pink" for people, "green" for bicycle and more
* Visualize an image, and add an overlay of colors on various regions

In [0]:
class DeepLabModel(object):
  """Class to load deeplab model and run inference."""

  INPUT_TENSOR_NAME = 'ImageTensor:0'
  OUTPUT_TENSOR_NAME = 'SemanticPredictions:0'
  INPUT_SIZE = 513
  FROZEN_GRAPH_NAME = 'frozen_inference_graph'

  def __init__(self, tarball_path):
    """Creates and loads pretrained deeplab model."""
    self.graph = tf.Graph()

    graph_def = None
    # Extract frozen graph from tar archive.
    tar_file = tarfile.open(tarball_path)
    for tar_info in tar_file.getmembers():
      if self.FROZEN_GRAPH_NAME in os.path.basename(tar_info.name):
        file_handle = tar_file.extractfile(tar_info)
        graph_def = tf.GraphDef.FromString(file_handle.read())
        break

    tar_file.close()

    if graph_def is None:
      raise RuntimeError('Cannot find inference graph in tar archive.')

    with self.graph.as_default():
      tf.import_graph_def(graph_def, name='')

    self.sess = tf.Session(graph=self.graph)

  def run(self, image):
    """Runs inference on a single image.

    Args:
      image: A PIL.Image object, raw input image.

    Returns:
      resized_image: RGB image resized from original input image.
      seg_map: Segmentation map of `resized_image`.
    """
    width, height = image.size
    resize_ratio = 1.0 * self.INPUT_SIZE / max(width, height)
    target_size = (int(resize_ratio * width), int(resize_ratio * height))
    resized_image = image.convert('RGB').resize(target_size, Image.ANTIALIAS)
    batch_seg_map = self.sess.run(
        self.OUTPUT_TENSOR_NAME,
        feed_dict={self.INPUT_TENSOR_NAME: [np.asarray(resized_image)]})
    seg_map = batch_seg_map[0]
    return resized_image, seg_map


def create_pascal_label_colormap():
  """Creates a label colormap used in PASCAL VOC segmentation benchmark.

  Returns:
    A Colormap for visualizing segmentation results.
  """
  colormap = np.zeros((256, 3), dtype=int)
  ind = np.arange(256, dtype=int)

  for shift in reversed(range(8)):
    for channel in range(3):
      colormap[:, channel] |= ((ind >> channel) & 1) << shift
    ind >>= 3

  return colormap


def label_to_color_image(label):
  """Adds color defined by the dataset colormap to the label.

  Args:
    label: A 2D array with integer type, storing the segmentation label.

  Returns:
    result: A 2D array with floating type. The element of the array
      is the color indexed by the corresponding element in the input label
      to the PASCAL color map.

  Raises:
    ValueError: If label is not of rank 2 or its value is larger than color
      map maximum entry.
  """
  if label.ndim != 2:
    raise ValueError('Expect 2-D input label')

  colormap = create_pascal_label_colormap()

  if np.max(label) >= len(colormap):
    raise ValueError('label value too large.')

  return colormap[label]


def vis_segmentation(image, seg_map):
    """Visualizes input image, segmentation map and overlay view."""
    LABEL_NAMES = np.asarray([
        'background', 'aeroplane', 'bicycle', 'bird', 'boat', 'bottle', 'bus',
        'car', 'cat', 'chair', 'cow', 'diningtable', 'dog', 'horse', 'motorbike',
        'person', 'pottedplant', 'sheep', 'sofa', 'train', 'tv'
    ])
    FULL_LABEL_MAP = np.arange(len(LABEL_NAMES)).reshape(len(LABEL_NAMES), 1)
    FULL_COLOR_MAP = label_to_color_image(FULL_LABEL_MAP)

    plt.figure(figsize=(15, 5))
    grid_spec = gridspec.GridSpec(1, 4, width_ratios=[6, 6, 6, 1])

    plt.subplot(grid_spec[0])
    plt.imshow(image)
    plt.axis('off')
    plt.title('input image')

    plt.subplot(grid_spec[1])
    seg_image = label_to_color_image(seg_map).astype(np.uint8)
    plt.imshow(seg_image)
    plt.axis('off')
    plt.title('segmentation map')

    plt.subplot(grid_spec[2])
    plt.imshow(image)
    plt.imshow(seg_image, alpha=0.7)
    plt.axis('off')
    plt.title('segmentation overlay')

    unique_labels = np.unique(seg_map)
    ax = plt.subplot(grid_spec[3])
    plt.imshow(
        FULL_COLOR_MAP[unique_labels].astype(np.uint8), interpolation='nearest')
    ax.yaxis.tick_right()
    plt.yticks(range(len(unique_labels)), LABEL_NAMES[unique_labels])
    plt.xticks([], [])
    ax.tick_params(width=0.0)
    plt.grid('off')
    plt.show()

## Select a pretrained model
We have trained the DeepLab model using various backbone networks. Select one from the MODEL_NAME list.

In [33]:
def deeplab_init():
    MODEL_NAME = 'mobilenetv2_coco_voctrainaug'
    # ['mobilenetv2_coco_voctrainaug', 'mobilenetv2_coco_voctrainval', 'xception_coco_voctrainaug', 'xception_coco_voctrainval']

    _DOWNLOAD_URL_PREFIX = 'http://download.tensorflow.org/models/'
    _MODEL_URLS = {
        'mobilenetv2_coco_voctrainaug':
            'deeplabv3_mnv2_pascal_train_aug_2018_01_29.tar.gz',
        'mobilenetv2_coco_voctrainval':
            'deeplabv3_mnv2_pascal_trainval_2018_01_29.tar.gz',
        'xception_coco_voctrainaug':
            'deeplabv3_pascal_train_aug_2018_01_04.tar.gz',
        'xception_coco_voctrainval':
            'deeplabv3_pascal_trainval_2018_01_04.tar.gz',
    }
    _TARBALL_NAME = 'deeplab_model.tar.gz'

    model_dir = tempfile.mkdtemp()
    tf.gfile.MakeDirs(model_dir)

    download_path = os.path.join(model_dir, _TARBALL_NAME)
    print('downloading model, this might take a while...')
    urllib.request.urlretrieve(_DOWNLOAD_URL_PREFIX + _MODEL_URLS[MODEL_NAME],
                               download_path)
    print('download completed! loading DeepLab model...')

    MODEL = DeepLabModel(download_path)
    print('model loaded successfully!')
    return MODEL


downloading model, this might take a while...
download completed! loading DeepLab model...
model loaded successfully!


In [0]:
def run_visualization(url, MODEL):
    """Inferences DeepLab model and visualizes result."""
    try:
        f = urllib.request.urlopen(url)
        jpeg_str = f.read()
        original_im = Image.open(BytesIO(jpeg_str))
    except IOError:
        print('Cannot retrieve image. Please check url: ' + url)
        return

    print('Image %s' % url)
    resized_im, seg_map = MODEL.run(original_im)
#   vis_segmentation(resized_im, seg_map)
    return resized_im, seg_map

In [0]:
def sd(a,b,nbits=8):
  return (a<<nbits)+b

def append_numbers(arr):
  return reduce(sd,arr)

def unappend_numbers(num):
  return list(map(lambda v:v&0xff,[num>>16,num>>8,num]))

def colour_strip(colour_pixel_rgb,shape=(30,120)):
  y=np.array(colour_pixel_rgb,dtype='uint8').reshape(1,-1)
  return PIL.Image.fromarray(np.full((shape[0],shape[1],3),y))

def inc_dict(dct,key):
  dct[key] = dct.get(key,0)+1

def get_dominant_obj_code(dct):
  return list(map(lambda i: i[0],sorted(dct.items(), key=operator.itemgetter(1),reverse=True)))

def dominant_obj_colours(img,mask,hist,bcg_obj=0,bcg_colour=[0,0,0]): #0 is black
  colours = {}
  bcg_colour = np.array(bcg_colour)
  img_arr = np.array(img)
  hc = hist.copy()
  hc[bcg_obj]=0
  dom_obj_data =[]
  dominant_obj_code = get_dominant_obj_code(hc)[0]
  nrows,ncols = mask.shape
  for row in range(nrows):
    for col in range(ncols):
      if mask[row,col]==dominant_obj_code:
        pix = img_arr[row,col]
        index = append_numbers(pix)
        colours[index] = colours.get(index,0)+1
        dom_obj_data.append(pix)
  return colours,dom_obj_data

def dom_colour(colours):
  dct = colours
  dc = sorted(dct.items(), key=operator.itemgetter(1),reverse=True)[0]
  dominant_colour = {'hex':dc[0],'rgb':unappend_numbers(dc[0])}
  print 'Most dominant colour','#'+str(dominant_colour['hex'])
  return colour_strip(dominant_colour['rgb'])

def dom_colour_range(dom_obj_data,n_clusters=10):
  km = KMeans(n_clusters=n_clusters)
  km.fit(dom_obj_data)

  cluster_num = list(map(int,km.predict(dom_obj_data)))
  cluster_sizes = list(map(cluster_num.count,set(cluster_num)))
  cluster_centers = list(km.cluster_centers_)
  data = dict(zip(cluster_sizes,cluster_centers))
  
  sk = data.keys()
  sk.sort(reverse=True)
  cl_centers = [ data[e] for e in sk]
  
  print 'Most dominant colour range','#'+hex(append_numbers(list(map(int,cl_centers[0]) )))
  return colour_strip(cl_centers[0])

def colour_ranges(dom_obj_data,n_clusters=10,echo=False):
  km = KMeans(n_clusters=n_clusters)
  km.fit(dom_obj_data)

  cluster_num = list(map(int,km.predict(dom_obj_data)))
  cluster_sizes = list(map(cluster_num.count,set(cluster_num)))
    
  cluster_centers = list(km.cluster_centers_)
  s = float(sum(cluster_sizes))
  percentages = list(map(lambda v:int(round((100*v)/s)),cluster_sizes))
  
  data = pd.DataFrame({'cluster_sizes':cluster_sizes,'cluster_centers':cluster_centers,'percentages':percentages})
  data.sort_values(by=['cluster_sizes'],ascending=False)
  cluster_sizes = list(data.loc[:,'cluster_sizes'])
  cl_centers = list(data.loc[:,'cluster_centers'])
  percentages = list(data.loc[:,'percentages'])
  
  cl_centers = list(map(lambda e: list(map(int,e)),cl_centers))
  if echo:
    print 'Ratio of colour ranges',ratio
  
  
  images = list(map(colour_strip,cl_centers))
  widths, heights = zip(*(i.size for i in images))

  max_width = max(widths)
  total_height = sum(heights)

  new_im = Image.new('RGB', (max_width, total_height))

  y_offset = 0
  if echo:
    print 'colour ranges sorted by dominance. (High to Low)'
    
  for im in images:
    new_im.paste(im, (0,y_offset))
    y_offset += im.size[1]
  return new_im,percentages,cl_centers


def dstsq(a,b,i):
  return (a[i]-b[i])**2

def euclidean(hex1,hex2):
  c1,c2 = list(map(unappend_numbers,[hex1,hex2]))
  return sum(list(map( lambda i: dstsq(c1,c2,i),list(range(0,3)))))

def rgb2hexstr(rgb_val):
  return '#'+hex(append_numbers(rgb_val)).replace('0x','').rjust(6,'0')

def get_colour_name(colour_rgb):
  valindex = colours_tree.query(colour_rgb)[1]
  rgb_val = web_colours_rgb[valindex]
  hexcode = rgb2hexstr(rgb_val)
  #hexcode = hexcode.replace('L','')
  colour_name = webcolors.css3_hex_to_names[hexcode]
  for k,v in replacements.items():
    colour_name = colour_name.replace(k,v)
  return str(colour_name)

In [0]:
def get_top_dominant_colours(image_url):
  hist = {}
  image,mask = run_visualization(image_url)
  n_clusters=3

  list(map( lambda imrow: list(map(lambda k: inc_dict(hist,k),imrow)),mask))
  colours,dom_obj_data = dominant_obj_colours(image,mask,hist)

  _,percentages,cl_centers = colour_ranges(dom_obj_data,n_clusters)
  colour_names = list(map(get_colour_name,cl_centers))
  
  cl_centers_hex = list(map(rgb2hexstr,cl_centers))
  return cl_centers_hex,colour_names,percentages

def get_top_dominant_colours_json(image_data):
  image_id,image_url,image_description = image_data
  cl_centers_hex,colour_names,percentages = get_top_dominant_colours(image_url)
  json_data = {'image':image_url,
               'description':image_description,
               'id':image_id}
  
  for i in range(len(cl_centers_hex)):
    cdata = {'colour-'+str(i+1):{'name':colour_names[i],
                                 'hex':cl_centers_hex[i],
                                 'percentage':percentages[i]}}
    json_data.update(cdata)
  
  return json_data

In [0]:
def process_file(input_filename,output_filename='output.json'):
  global web_colours_rgb,colours_tree,replacements
  web_colours_rgb = list(map(unappend_numbers,list(map(lambda v:int(v.replace('#','0x'),0),webcolors.css3_hex_to_names.keys()))))
  colours_tree = spatial.KDTree(web_colours_rgb)
  replacements = {'light':'light ','dark':'dark ','dim':'dark '}

  try:
    file = open(input_filename,'r')
    data = []
    for line in file:
      dt = line.split('|')
      if len(dt)>1:
        dt = dt[1:]
        data.append(dt)
    file.close()
    
    jsons = list(map(get_top_dominant_colours_json,data))
    final_data = {}
    for j in jsons:
      final_data.update({j['id']:j})
    
    print(final_data)
    
    with open(output_filename,'w') as of:
      json.dump(final_data,of)
    
  except Exception as e:
    print(e)
    
def main():
  if len(sys.argv)<2:
    print('No input file specified!')
    return
  
  input_file = sys.argv[1]
  output_file = 'output.json'
  if len(sys.argv)>2:
    output_file = sys.argv[2]
  
  process_file(input_file,output_file)
  
if __name__ == "__main__":
  main()

In [0]:
# sys.argv = ['file.py','tkc.txt']

In [0]:
# main()

In [0]:
############################################################################################3
############################################################################################3
############################################################################################3

In [5]:
import os
import sys
import PIL
from io import BytesIO
from six.moves import urllib
import numpy as np
from PIL import Image
import json
import cv2

def unappend_numbers(num):
  return list(map(lambda v:v&0xff,[num>>16,num>>8,num]))

def impose_json_on_image(json_data):
  try:
    url = json_data['image']
    colours = json_data['colour-1'],json_data['colour-2'],json_data['colour-3']
    strs_to_write = list(map(lambda c:str(c['name'])+'('+str(c['percentage'])+')',colours))
  
    print(url)
    fa = urllib.request.urlopen(url)
    jpeg_str = fa.read()
    original_im = Image.open(BytesIO(jpeg_str))
    img = cv2.cvtColor(np.array(original_im), cv2.COLOR_RGB2BGR)
    for i in range(len(strs_to_write)):
      c_rgb = tuple(unappend_numbers(int(colours[i]['hex'].replace('#','0x'),0)))
      cv2.rectangle(img,(0,10*i), (10,(10*i)+10), c_rgb, cv2.FILLED)
      cv2.putText(img, strs_to_write[i], (10, (i*10)+10), cv2.FONT_HERSHEY_SIMPLEX, 0.25, (0, 0, 0), 1, cv2.LINE_AA)
    cv2.imwrite(json_data['id']+'.jpg',img)
    fa.close()
  except Exception as e:
    exc_type, exc_obj, exc_tb = sys.exc_info()
    print(e,exc_tb.tb_lineno)
    return

def visualize_json(json_file):
  try:
    with open(json_file,'r') as f:    
      data = json.load(f)
      for k in data.keys():
        impose_json_on_image(data[k])
  except Exception as e:
    exc_type, exc_obj, exc_tb = sys.exc_info()
    print(e,exc_tb.tb_lineno,json_file)

def main():
  opf = ''
  if len(sys.argv)<2:
    print('Assuming json file to be output.json')
    opf = 'output.json'
  else:
    opf = sys.argv[1]
  opf = 'output.json'

if __name__=="__main__":
  main()

http://www.picalike.com/train/export/5/50088.jpg
http://www.picalike.com/train/export/22/228054.jpg
http://www.picalike.com/train/export/22/228057.jpg
http://www.picalike.com/train/export/22/228056.jpg


In [8]:
!python top_colours.py tkc.txt

downloading model, this might take a while...
download completed! loading DeepLab model...
2019-05-12 18:21:37.051080: I tensorflow/core/platform/profile_utils/cpu_utils.cc:94] CPU Frequency: 2300000000 Hz
2019-05-12 18:21:37.051342: I tensorflow/compiler/xla/service/service.cc:150] XLA service 0x55dc53bde6e0 executing computations on platform Host. Devices:
2019-05-12 18:21:37.051379: I tensorflow/compiler/xla/service/service.cc:158]   StreamExecutor device (0): <undefined>, <undefined>
model loaded successfully!
Image http://www.picalike.com/train/export/5/50088.jpg
Image http://www.picalike.com/train/export/22/228054.jpg
Image http://www.picalike.com/train/export/22/228057.jpg
Image http://www.picalike.com/train/export/22/228056.jpg
{'50088': {'colour-3': {'percentage': 26, 'hex': '#323232', 'name': 'dark slategray'}, 'colour-2': {'percentage': 61, 'hex': '#656566', 'name': 'dark grey'}, 'colour-1': {'percentage': 12, 'hex': '#ababa8', 'name': 'dark gray'}, 'description': 'fashion_s

In [11]:
!python visualize.py

Assuming json file to be output.json
http://www.picalike.com/train/export/5/50088.jpg
http://www.picalike.com/train/export/22/228054.jpg
http://www.picalike.com/train/export/22/228057.jpg
http://www.picalike.com/train/export/22/228056.jpg


In [0]:
############################################################################################3
############################################################################################3
############################################################################################3

In [0]:
# cdict = {    "white":0xffffff, 
#     "beige" :0xF5F5DC,
#     "yellow":0xfff020,
#     "gold" :0xFFD700,
#     "orange" :0xFFA500,
#     "red":0xff0d0d, 
#     "rose" :0xFF007F,
#     "pink" :0xFFC0CB,
#     "violet":0x9932CC,
#     "dark brown" :0x8B4513,
#     "light brown" :0xDEB887, 
#     "light green":0x7CFC00,  
#     "dark green" :0x008000,
#     "olive" :0x808000,
#     "turquoise" :0x40E0D0,
#     "light blue" :0x87CEFA, 
#     "dark blue" :0x00008B,
#     "silver" :0xC0C0C0,
#     "light grey" :0xDCDCDC,
#     "dark grey" :0x696969,
#     "black":0x000000
#         }  
  
# wanted_colours = {'dimgrey':'darkgrey','lightslategrey':'lightgrey','grey':'lightgrey','darkslategray':'darkgrey',
#         'sandybrown':'lightbrown','saddlebrown':'darkbrown','rosebrown':'darkbrown'}
# colour_names = list(map(lambda s: s.replace(' ',''), cdict.keys()))
# for c in colour_names:
#   wanted_colours[c] = c 
# main_colours_rgb = np.array(list(map(unappend_numbers,cdict.values())))

In [0]:
#     "white"=>"ffffff", 
#     "beige" => "F5F5DC",
#     "yellow"=>"fff020",
#     "gold" => "FFD700",
#     "orange" => "FFA500",
#     "red"=>"ff0d0d", 
#     "rose" => "FF007F",
#     "pink" => "FFC0CB",
#     "violet"=>"9932CC",
#     "dark brown" => "8B4513",
#     "light brown" => "DEB887", 
#     "light green"=>"7CFC00",  
#     "dark green" => "008000",
#     "olive" => "808000",
#     "turquoise" => "40E0D0",
#     "light blue" => "87CEFA", 
#     "dark blue" => "00008B",
#     "silver" => "C0C0C0",
#     "light grey" => "DCDCDC",
#     "dark grey" => "696969",
#     "black"=>"000000",


# 1|50088|http://www.picalike.com/train/export/5/50088.jpg|fashion_shoe_outdoor-shoes_outdoor-sneaker
# 2|228054|http://www.picalike.com/train/export/22/228054.jpg|fashion_accessoires_necklace
# 3|228057|http://www.picalike.com/train/export/22/228057.jpg|fashion_shoe_boots_ankle-boots
# 4|228056|http://www.picalike.com/train/export/22/228056.jpg|furniture_seating_couch
# 5|228051|http://www.picalike.com/train/export/22/228051.jpg|fashion_top_blouse
# 6|228050|http://www.picalike.com/train/export/22/228050.jpg|fashion_bag
# 7|228053|http://www.picalike.com/train/export/22/228053.jpg|fashion_leg_skirt
# 8|203184|http://www.picalike.com/train/export/20/203184.jpg|fashion_top_beachware_bikini
# 9|228059|http://www.picalike.com/train/export/22/228059.jpg|fashion_top_tunic
# 10|228058|http://www.picalike.com/train/export/22/228058.jpg|fashion_top_jacket
# 11|89370|http://www.picalike.com/train/export/8/89370.jpg|fashion_bag
# 12|89371|http://www.picalike.com/train/export/8/89371.jpg|furniture_lamp_floor-lamp_three-legged-floorlamp
# 13|89372|http://www.picalike.com/train/export/8/89372.jpg|fashion_shoe_outdoor-shoes_outdoor-boots
# 14|89373|http://www.picalike.com/train/export/8/89373.jpg|fashion_top
# 15|89374|http://www.picalike.com/train/export/8/89374.jpg|fashion_shoe_soccer-shoes
# 16|89375|http://www.picalike.com/train/export/8/89375.jpg|fashion_top_jacket
# 17|89376|http://www.picalike.com/train/export/8/89376.jpg|fashion_top_bolero
# 18|89377|http://www.picalike.com/train/export/8/89377.jpg|fashion_top_beachware_swimsuit
# 19|89378|http://www.picalike.com/train/export/8/89378.jpg|fashion_top_blouse
# 20|89379|http://www.picalike.com/train/export/8/89379.jpg|fashion_leg_skirt_jeans-skirt
# 21|202448|http://www.picalike.com/train/export/20/202448.jpg|fashion_top_vest
# 22|5988|http://www.picalike.com/train/export/0/5988.jpg|fashion_top_jacket
# 23|5989|http://www.picalike.com/train/export/0/5989.jpg|fashion_top_jacket
# 24|5982|http://www.picalike.com/train/export/0/5982.jpg|fashion_top_jacket
# 25|5983|http://www.picalike.com/train/export/0/5983.jpg|fashion_accessoires_watch
# 26|5980|http://www.picalike.com/train/export/0/5980.jpg|fashion_leg_skirt
# 27|5981|http://www.picalike.com/train/export/0/5981.jpg|fashion_shoe_sneaker
# 28|5986|http://www.picalike.com/train/export/0/5986.jpg|fashion_accessoires_scarf
# 29|5987|http://www.picalike.com/train/export/0/5987.jpg|fashion_top_button-down-shirt_business-shirt
# 30|5984|http://www.picalike.com/train/export/0/5984.jpg|fashion_top_underwear_bra
# 31|5985|http://www.picalike.com/train/export/0/5985.jpg|fashion_top_blouse
# 32|79236|http://www.picalike.com/train/export/7/79236.jpg|fashion_shoe_pumps
# 33|194080|http://www.picalike.com/train/export/19/194080.jpg|fashion_top_dress
# 34|79237|http://www.picalike.com/train/export/7/79237.jpg|fashion_accessoires_glasses_sunglasses_wayfarer-sunglasses
# 35|194081|http://www.picalike.com/train/export/19/194081.jpg|fashion_bag_bagpack
# 36|97159|http://www.picalike.com/train/export/9/97159.jpg|fashion_accessoires_ring
# 37|97158|http://www.picalike.com/train/export/9/97158.jpg|fashion_top_tunic
# 38|97153|http://www.picalike.com/train/export/9/97153.jpg|fashion_accessoires_ring
# 39|97152|http://www.picalike.com/train/export/9/97152.jpg|fashion_leg_shorts_jeans-shorts
# 40|97151|http://www.picalike.com/train/export/9/97151.jpg|fashion_accessoires_bangle
# 41|97150|http://www.picalike.com/train/export/9/97150.jpg|fashion_shoe_pumps
# 42|97157|http://www.picalike.com/train/export/9/97157.jpg|fashion_accessoires_belt
# 43|97156|http://www.picalike.com/train/export/9/97156.jpg|fashion_shoe_boots_high-boots
# 44|97155|http://www.picalike.com/train/export/9/97155.jpg|fashion_top_button-down-shirt_casual-shirt
# 45|97154|http://www.picalike.com/train/export/9/97154.jpg|fashion_top_blouse
# 46|194082|http://www.picalike.com/train/export/19/194082.jpg|fashion_accessoires_bangle
# 47|270|http://www.picalike.com/train/export/0/270.jpg|fashion_leg_shorts_jeans-shorts
# 48|271|http://www.picalike.com/train/export/0/271.jpg|fashion_top_blouse
# 49|272|http://www.picalike.com/train/export/0/272.jpg|fashion_top_underwear_bra
# 50|273|http://www.picalike.com/train/export/0/273.jpg|fashion_leg_pants
# 51|274|http://www.picalike.com/train/export/0/274.jpg|furniture_closet_sideboard
# 52|275|http://www.picalike.com/train/export/0/275.jpg|fashion_accessoires_glasses_sunglasses_wayfarer-sunglasses
# 53|276|http://www.picalike.com/train/export/0/276.jpg|fashion_accessoires_gloves
# 54|277|http://www.picalike.com/train/export/0/277.jpg|fashion_leg_shorts_no-jeans-shorts
# 55|278|http://www.picalike.com/train/export/0/278.jpg|fashion_headgear_winter-hat
# 56|279|http://www.picalike.com/train/export/0/279.jpg|furniture_seating_bean-bag
# 57|16707|http://www.picalike.com/train/export/1/16707.jpg|fashion_top_woman-top
# 58|16706|http://www.picalike.com/train/export/1/16706.jpg|furniture_lamp_table-lamp
# 59|16701|http://www.picalike.com/train/export/1/16701.jpg|fashion_top_beachware_swimsuit
# 60|16700|http://www.picalike.com/train/export/1/16700.jpg|fashion_top_poloshirt
# 61|16703|http://www.picalike.com/train/export/1/16703.jpg|fashion_top_jacket
# 62|16702|http://www.picalike.com/train/export/1/16702.jpg|fashion_top_beachware_swimsuit
# 63|88023|http://www.picalike.com/train/export/8/88023.jpg|fashion_top_blouse
# 64|88022|http://www.picalike.com/train/export/8/88022.jpg|fashion_leg_pants_sweatpants
# 65|88021|http://www.picalike.com/train/export/8/88021.jpg|fashion_top_coat
# 66|88020|http://www.picalike.com/train/export/8/88020.jpg|furniture_sleeping-possibilities_bed_single-bed
# 67|88027|http://www.picalike.com/train/export/8/88027.jpg|fashion_top_blouse
# 68|88026|http://www.picalike.com/train/export/8/88026.jpg|fashion_top_jacket
# 69|88025|http://www.picalike.com/train/export/8/88025.jpg|fashion_top_beachware_bikini
# 70|88024|http://www.picalike.com/train/export/8/88024.jpg|furniture_sleeping-possibilities_bed_childrens-bed
# 71|54547|http://www.picalike.com/train/export/5/54547.jpg|fashion_leg_skirt
# 72|194083|http://www.picalike.com/train/export/19/194083.jpg|fashion_shoe_sandal
# 73|88029|http://www.picalike.com/train/export/8/88029.jpg|fashion_top_suit
# 74|88028|http://www.picalike.com/train/export/8/88028.jpg|fashion_accessoires_earring
# 75|42085|http://www.picalike.com/train/export/4/42085.jpg|home-textiles_window-blinds_curtains
# 76|79232|http://www.picalike.com/train/export/7/79232.jpg|fashion_shoe_boots_low-boots
# 77|101359|http://www.picalike.com/train/export/10/101359.jpg|fashion_leg
# 78|119869|http://www.picalike.com/train/export/11/119869.jpg|fashion_bag_bagpack
# 79|119868|http://www.picalike.com/train/export/11/119868.jpg|fashion_top_blouse
# 80|119865|http://www.picalike.com/train/export/11/119865.jpg|fashion_top_blouse
# 81|119864|http://www.picalike.com/train/export/11/119864.jpg|fashion_shoe_boots_rubber-boots
# 82|119867|http://www.picalike.com/train/export/11/119867.jpg|fashion_top_poloshirt
# 83|119866|http://www.picalike.com/train/export/11/119866.jpg|fashion_top_woman-top
# 84|119861|http://www.picalike.com/train/export/11/119861.jpg|fashion_leg_skirt
# 85|119860|http://www.picalike.com/train/export/11/119860.jpg|fashion_bag_bagpack
# 86|119863|http://www.picalike.com/train/export/11/119863.jpg|fashion_shoe_boots_ankle-boots
# 87|119862|http://www.picalike.com/train/export/11/119862.jpg|fashion_shoe_pumps
# 88|80278|http://www.picalike.com/train/export/8/80278.jpg|fashion_accessoires_belt
# 89|194084|http://www.picalike.com/train/export/19/194084.jpg|vehicles_bike_mountainbike
# 90|80279|http://www.picalike.com/train/export/8/80279.jpg|fashion_top_dress
# 91|79233|http://www.picalike.com/train/export/7/79233.jpg|vehicles_bike_mountainbike
# 92|176062|http://www.picalike.com/train/export/17/176062.jpg|fashion_shoe_sandals
# 93|80276|http://www.picalike.com/train/export/8/80276.jpg|fashion_top_underwear_corsage
# 94|176060|http://www.picalike.com/train/export/17/176060.jpg|furniture_seating_couch
# 95|176061|http://www.picalike.com/train/export/17/176061.jpg|fashion_top_dress
# 96|49569|http://www.picalike.com/train/export/4/49569.jpg|fashion_top_dress
# 97|49568|http://www.picalike.com/train/export/4/49568.jpg|fashion_shoe_ballerinas
# 98|176064|http://www.picalike.com/train/export/17/176064.jpg|fashion_top_dress
# 99|80277|http://www.picalike.com/train/export/8/80277.jpg|fashion_leg_jumpsuit
# 100|49565|http://www.picalike.com/train/export/4/49565.jpg|fashion_headgear_cap-with-screen